In [1]:
import pandas as pd

### Clean up the private school text file mess

## Load the file
with open('TEMP\private_schools raw data.csv', 'r') as f:
    lines = f.readlines()

## Clean up the lines
lines = [line.strip() for line in lines]

#put the lines into a dataframe
df = pd.DataFrame(lines)



# drop rows with any of the strings in remove
remove = ['Homes for sale', 'Reviews', 'Currently unrated', 'N/A']

df = df[~df[0].str.contains('|'.join(remove))]
df = df[~df[0].str.contains('Private Schools in')]


df.info()
df.head(40)

# add a header row
df.columns = ['School']

# reindex the dataframe
df = df.reset_index(drop=True)

## Take every other row and move it to the previous row as a new column
df['Address'] = df['School'].shift(-1)
# remove the rows with the address

df = df[::2]
df = df.reset_index(drop=True)

# Get the city name out of the address column
df['City'] = df['Address'].str.split(',').str[1]
df['City'] = df['City'].str.strip()

# Get the state name out of the address column
df['State'] = df['Address'].str.split(',').str[2]
df['State'] = df['State'].str.strip()

# Get the zip code out of the address column
df['Zip'] = df['Address'].str.split(',').str[3]
df['Zip'] = df['Zip'].str.strip()

# drop everything after the first ',' in the address column
df['Address'] = df['Address'].str.split(',').str[0]

### Add the city name to the front of the school name
# this may help in the matching process because most of the mhsaa list has the town name in the school name

df['School'] = df['City'] + ' - ' + df['School']

## Replace the words 'High School' with 'HS' in the school name

df['School'] = df['School'].str.replace('High School', 'HS')



# df.head()

# ## Find out how many schools are in each city

# df['City'].value_counts()

# ## Find out how many school names have any of these characters '&','-','.'
# chars = ['&','-']


# df['School'].str.contains('|'.join(chars)).value_counts()

# # print those schools that have any of the characters
# # df[df['School'].str.contains('|'.join(chars))]

<class 'pandas.core.frame.DataFrame'>
Int64Index: 806 entries, 0 to 2178
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       806 non-null    object
dtypes: object(1)
memory usage: 12.6+ KB


False    390
True      13
Name: School, dtype: int64

In [3]:

# ### Output the dataframe to a csv file
# df.to_csv('TEMP\private_schools.csv', index=False)


In [4]:
## Use fuzzywuzzy to find the best match for the school name

import fuzzywuzzy
from fuzzywuzzy import process

# get the list of school names

school_names = df['School'].unique()

#### get the list of mhsaa schools to compare to

# load the mhsaa schools file
mhsaa = pd.read_csv('data\school_info\mhsaa_enrolment_2022.csv')   

# get the list of mhsaa school names
mhsaa_names = mhsaa['school_name'].unique()

## compair the mhsaa school names to the private school names
## set the match threadhold to 90% to try to elimate error matches
# only about 15-20% of the mhsaa names should have strong matches within the private schools


# create a new dataframe to hold the matches
matches = pd.DataFrame(columns=['mhsaa_name', 'private_name', 'score'])

# loop through the mhsaa school names
for mhsaa_name in mhsaa_names:
    # find the best match for the mhsaa school name in the private school names
    match = process.extractOne(mhsaa_name, school_names, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    # if the match score is greater than 90% add the match to the dataframe
    if match[1] > 75:
        matches = matches.append({'mhsaa_name': mhsaa_name, 'private_name': match[0], 'score': match[1]}, ignore_index=True)


matches.info()




C:\Users\Justin\AppData\Local\Temp\ipykernel_20452\3879399148.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matches = matches.append({'mhsaa_name': mhsaa_name, 'private_name': match[0], 'score': match[1]}, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_20452\3879399148.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matches = matches.append({'mhsaa_name': mhsaa_name, 'private_name': match[0], 'score': match[1]}, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_20452\3879399148.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matches = matches.append({'mhsaa_name': mhsaa_name, 'private_name': match[0], 'score': match[1]}, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ip

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   mhsaa_name    64 non-null     object
 1   private_name  64 non-null     object
 2   score         64 non-null     object
dtypes: object(3)
memory usage: 1.6+ KB


C:\Users\Justin\AppData\Local\Temp\ipykernel_20452\3879399148.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matches = matches.append({'mhsaa_name': mhsaa_name, 'private_name': match[0], 'score': match[1]}, ignore_index=True)
C:\Users\Justin\AppData\Local\Temp\ipykernel_20452\3879399148.py:32: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  matches = matches.append({'mhsaa_name': mhsaa_name, 'private_name': match[0], 'score': match[1]}, ignore_index=True)


In [10]:
matches.head(100)


### Output the dataframe to a csv file
matches.to_csv('TEMP\private_schools.csv', index=False)


### 1/10/23

## Just finishes looking through the result of the 75 TH match
# there are some errors in the matches from 75-79

## There are also some schools that are obviously missing like Lansing Catholic. This probably
## means the list of private schools is incomplete. Might utimately need to find a better source


### working skelton but needs lots of refining
# filtering out a lot of the small schools like the islamic ones from the private school list is something to try

# Also should replace High School in private names with HS to match the mhsaa names
    # Just did this. first test on 75 TH returns 64 results
    # That is 64/599 = 10.7% of the mhsaa schools
    # Expected results are 10-20% of the mhsaa schools